# **3d Solids**

## Step 1 : Mesh

In [7]:
from dewloosh.mesh.rgrid import gridH8 as grid
import numpy as np

size = Lx, Ly, Lz = (20, 4, 4)
shape = nx, ny, nz = (40, 10, 10)

gridparams = {
    'size': size,
    'shape': shape,
    'origo': (0, 0, 0),
    'start': 0
}

coords, topo = grid(**gridparams)

## Step 2 : Material

In [8]:
E = 12000.
nu = 0.2
A = np.array([
    [1, nu, nu, 0, 0, 0], 
    [nu, 1, nu, 0, 0, 0],
    [nu, nu, 1, 0, 0, 0], 
    [0., 0, 0, (1-nu)/2, 0, 0],
    [0., 0, 0, 0, (1-nu)/2, 0],
    [0., 0, 0, 0, 0, (1-nu)/2]]) * (E / (1-nu**2))


## Step 3 : Boundary Conditions

In [9]:
# fix points at x==0
cond = coords[:, 0] <= 0.001
ebcinds = np.where(cond)[0]
fixity = np.zeros((coords.shape[0], 6), dtype=bool)
fixity[ebcinds, :] = True
fixity[:, 3:] = True

# unit vertical load at (Lx, Ly)
F = 10
cond = (coords[:, 0] > (Lx-(1e-12))) & \
    (coords[:, 1] > (Ly-(1e-12))) & \
        (coords[:, 2] > (Lz-(1e-12)))
nbcinds = np.where(cond)[0]
loads = np.zeros((coords.shape[0], 6))
loads[nbcinds, 2] = -F


## Step 4 : Assembly and Solution

In [10]:
from sigmaepsilon.fem.elib import H8, Q4M as Q4

In [11]:
H8.dofs, H8.NDOFN, H8.Model, H8.Geom

AttributeError: type object 'H8' has no attribute 'Model'

In [ ]:
Q4.dofs, Q4.NDOFN, Q4.Model, Q4.Geom

(('UX', 'UY'), 2)

In [4]:
from sigmaepsilon import Structure, PointData, FemMesh
from sigmaepsilon.mesh.space import StandardFrame
from sigmaepsilon.fem.elib import H8 as CellData
from dewloosh.math.array import repeat

GlobalFrame = StandardFrame(dim=3)

# pointdata
pd = PointData(coords=coords, frame=GlobalFrame,
               loads=loads, fixity=fixity)

# celldata
frames = repeat(GlobalFrame.show(), topo.shape[0])
cd = CellData(topo=topo, frames=frames)

# set up mesh and structure
mesh = FemMesh(pd, cd, model=A, frame=GlobalFrame)
structure = Structure(mesh=mesh)

ValueError: cannot reshape array of size 0 into shape (0,1)

In [ ]:
structure.linsolve()
dofsol = structure.nodal_dof_solution()
structure.mesh.pointdata['x'] = coords + dofsol[:, :3]